In [1]:
#%%capture
%run ../dca/input/Format.ipynb
import ROOT as root
from array import array
import math
root.gErrorIgnoreLevel = root.kFatal
%jsroot on
root.gStyle.SetOptStat(0)

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


Error in <TUnixSystem::FindDynamicLibrary>: input/logo/PHENIXTools/lib/libLogoPainter.so does not exist in /home/yoren/bnl/ROOT/install/lib:.:/home/yoren/bnl/ROOT/install/lib:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/lib/x86_64-linux-gnu/tls/haswell/x86_64:/lib/x86_64-linux-gnu/tls/haswell:/lib/x86_64-linux-gnu/tls/x86_64:/lib/x86_64-linux-gnu/tls:/lib/x86_64-linux-gnu/haswell/x86_64:/lib/x86_64-linux-gnu/haswell:/lib/x86_64-linux-gnu/x86_64:/lib/x86_64-linux-gnu:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v3:/usr/lib/x86_64-linux-gnu/glibc-hwcaps/x86-64-v2:/usr/lib/x86_64-linux-gnu/tls/haswell/x86_64:/usr/lib/x86_64-linux-gnu/tls/haswell:/usr/lib/x86_64-linux-gnu/tls/x86_64:/usr/lib/x86_64-linux-gnu/tls:/usr/lib/x86_64-linux-gnu/haswell/x86_64:/usr/lib/x86_64-linux-gnu/haswell:/usr/lib/x86_64-linux-gnu/x86_64:/usr/lib/x86_64-linux-gnu:/lib/glibc-hwcaps/x86-64-v3:/lib/glibc-hwcaps/x86-64-v2:/lib/tls/haswell/x86_64:/lib/tls/haswell:/lib

In [2]:
root.gROOT.SetBatch(True)

In [3]:
root.gSystem.Load("input/WriteEvent_C.so")

0

In [4]:
# ---------------------------
# Input sample + generation settings
# ---------------------------

# Example:
# part index mapping (your list)
part_names = ["pi0","eta","rho->ee","omega","omega->pi0ee","eta'->gamma ee",#0-5
              "phi->ee","phi->etaee","jpsi","psi2s","ccbar","bbbar","qgp"]#6-12

part = 8  # qgp (thermal)
# Pick your input ROOT file (inside file_path)
fname = "star_jpsi_ee.root"   # example
#fname = "thermal.root"   # example

# Where the files live
file_path = "/media/yoren/T7_Shield/PHENIX/Dielectrons/helios/"  # adjust

# IMPORTANT: explicitly tell the notebook what kind of sample this is
#   - "helios_flat_pt" : generated flat in pT, needs pT reweight
#   - "thermal"        : already thermal, no pT reweight
SAMPLE_KIND = "helios_flat_pt"   # change to "thermal" when needed

# Generation ranges used when producing the sample (needed for correct normalization)
GEN_PT_MIN  = 0.0
GEN_PT_MAX  = 10.0
GEN_Y_MIN   = -1.0
GEN_Y_MAX   =  1.0
GEN_PHI_MIN = 0.0
GEN_PHI_MAX = 2.0*math.pi


# Reweight only if the input was generated with wrong pT spectrum
# - For thermal qgp: False
# - For HELIOS flat: True
# - For special pi0 case where you *know* gen spectrum TF1: True
DO_PT_REWEIGHT = (part != 12)

DO_SMAER_PT = True  # whether to apply SMAER pT reweighting (only for pi0 and eta)
DO_SINGLE_TRACK_CUT = True  # whether to apply single track cuts (rapidity and pT)

# For quick tests you can limit event count (set to None to run full)
nev_set = int(1e6)

print("DO_PT_REWEIGHT:", DO_PT_REWEIGHT)


DO_PT_REWEIGHT: True


In [5]:
import ROOT as root
import math

# ---- arrays used in hadron_yield construction ----
dNdy_pp = [42.2*95/257, 42.2*11/257, 8.6*42/257, 4.3, 4.3, 0.92, 0.93, 0.93, 1.77e-5, 0.133/1000/42.2, 0.12, 0.01, 10]
m_pp    = [0.1349766, 0.547862, 0.77526, 0.78265, 0.78265, 0.95778, 1.019461, 1.019461, 3.096916, 3.68609, -1.019461, -9.46030, 1]

Ncolls  = [292, 771, 282.4, 82.6, 12.1, 3]

Br_to_ee = [1.174e-2, 6.9e-3, 4.73e-5, 7.38e-5, 7.7e-4, 4.73e-4, 2.98e-4, 1.08e-4, 5.94e-2, 7.9e-3, 1, 1, 1]

hith_pt_ratio = [1.0, 0.48, 1.0, 0.78, 0.78, 0.25, 0.4, 0.4, 0.01, 0.5, 1, 1, 1]
pi0_high_pt_ratio = [0.00623224, 0.00228273, 0.000667683, 9.78082e-05, 2.425e-05]

RAA_jpsi = [0.4,0.2,0.3,0.7,0.8,0.8]

hag_params_try = [
  [ 573.1,  0.4178, 0.2118, 0.7282, 8.287],
  [1505.2,  0.4347, 0.2610, 0.7205, 8.319], # 0–20%
  [735.9,   0.4103, 0.1884, 0.7316, 8.272], # 20–40%
  [296.2,   0.4038, 0.1290, 0.7313, 8.207], # 40–60%
  [118.1,   0.4416, 0.0559, 0.7230, 8.163], # 60–80%
  [51.1,    0.2470, 0.0619, 0.7101, 8.459], # 80–93%
]

In [6]:
MPI0 = 0.134  # used in your mt shift

def Hagedorn_Yield_Func(x, p):
    pt   = x[0]
    mass = p[0]
    mt   = root.TMath.Sqrt(pt*pt + mass*mass - MPI0*MPI0)

    A  = p[1]
    a  = p[2]
    b  = p[3]
    p0 = p[4]
    n  = p[5]

    return 2.0*root.TMath.Pi() * pt * A * pow(root.TMath.Exp(-a*mt - b*mt*mt) + mt/p0, -n)

def fHagedorn(name, lowerlim, upperlim, mass=0.135, A=504.5, a=0.5169, b=0.1626, p0=0.7366, n=8.274):
    f = root.TF1(name, Hagedorn_Yield_Func, lowerlim, upperlim, 6)
    f.SetParameters(mass, A, a, b, p0, n)
    f.SetParNames("mass","A","a","b","p0","n")
    return f

In [7]:
# Output file for histograms
outfilename = "output/hists/" + fname.replace(".root", "_hists.root")
print("Output:", outfilename)


Output: output/hists/star_jpsi_ee_hists.root


In [8]:
f = root.TFile.Open(file_path+fname, "READ")
assert f and not f.IsZombie(), f"Could not open {fname}"

T = f.Get("T")
assert T, "Could not find TTree named 'T'"

print("Entries:", T.GetEntries())
T.Print()

Entries: 10000000
******************************************************************************
*Tree    :T         : Tree with my events                                    *
*Entries : 10000000 : Total =      3201031419 bytes  File  Size =  747573753 *
*        :          : Tree compression factor =   4.28                       *
******************************************************************************
*Branch  :MyEvent                                                            *
*Entries : 10000000 : BranchElement (see below)                              *
*............................................................................*
*Br    0 :fUniqueID : UInt_t                                                 *
*Entries : 10000000 : Total  Size=   40009532 bytes  File Size  =     184815 *
*Baskets :       99 : Basket Size=     879104 bytes  Compression= 216.47     *
*............................................................................*
*Br    1 :fBits     : UInt_t      

In [9]:
def get_tracks_from_event(MyEvent):
    """Return list of WriteTrack (by value) from a WriteEvent."""
    n = int(MyEvent.GetNEntries())
    return [MyEvent.GetWriteTrack(i) for i in range(n)]


In [10]:
def track_info(trk):
    """Extract common fields from WriteTrack."""
    pid = int(trk.GetID())
    isFinal = int(trk.GetFinal())
    br = float(trk.GetBranch())
    parent = int(trk.GetParentIndex())
    w = float(trk.GetWeight())
    p4 = root.TLorentzVector(float(trk.GetPx()), float(trk.GetPy()), float(trk.GetPz()), float(trk.GetEnergy()))
    return dict(pid=pid, isFinal=isFinal, br=br, parent=parent, w=w, p4=p4)


In [11]:
# single “centrality-free” choice
i0 = 0  # use the first row consistently

hadron_yield = fHagedorn("hadron_yield", 0.01, 25, mass=m_pp[part])

print("integral pre-scale:", hadron_yield.Integral(0.01, 20))

# default scaling: A *= 1.0 * Br_to_ee[part]
A0 = hadron_yield.GetParameter(1)
hadron_yield.SetParameter(1, A0 * 1.0 * Br_to_ee[part])

# if (part<2 || part==8): replace params with hag_params_try[i0], then A *= Br_to_ee[part]
if (part < 2) or (part == 8):
    c,a,b,p0,n = hag_params_try[i0]
    hadron_yield = fHagedorn("hadron_yield_new", 0.01, 25, mass=m_pp[part], A=c, a=a, b=b, p0=p0, n=n)
    A1 = hadron_yield.GetParameter(1)
    hadron_yield.SetParameter(1, A1 * Br_to_ee[part])

# if (part==8 || part==9): normalize integral and scale with dNdy * Br * RAA
# (since no Ncolls, we drop it; since no i, use RAA_jpsi[i0])
if (part == 8) or (part == 9):
    integ = hadron_yield.Integral(0.01, 10.0)
    if integ > 0:
        A = hadron_yield.GetParameter(1)
        hadron_yield.SetParameter(1, A / integ * dNdy_pp[part] * Br_to_ee[part] * RAA_jpsi[i0] * Ncolls[i0])

# if (part < 8 && part > 0): high-pt ratio scaling (use i0)
if (part < 8) and (part > 0):
    denom = (hadron_yield.Eval(5.0) / Br_to_ee[part] / pi0_high_pt_ratio[i0])
    if denom != 0:
        A = hadron_yield.GetParameter(1)
        hadron_yield.SetParameter(1, A * hith_pt_ratio[part] / denom)

print("integral post-scale:", hadron_yield.Integral(0.01, 25) / Br_to_ee[part])
print("eval(5)/BR:", hadron_yield.Eval(5.0) / Br_to_ee[part])

integral pre-scale: 0.03232622286778624
integral post-scale: 0.002068360546008644
eval(5)/BR: 3.430996614016027e-05


In [12]:
GEN_PT_MIN = 0.0
GEN_PT_MAX = 10.0

def pt_reweight(pt):
    # thermal qgp: no reweight
    if part == 12:
        return 1.0
    if pt < GEN_PT_MIN or pt > GEN_PT_MAX:
        return 0.0

    tgt = float(hadron_yield.Eval(pt))
    gen_flat = 1.0 / (GEN_PT_MAX - GEN_PT_MIN)
    return tgt / gen_flat


In [13]:
def single_track_cut(trk):
    """Rapidity cut of |y| < 1.0 and pT > 0.2 GeV."""
    p4 = trk
    y = p4.Eta()
    pt = p4.Pt()
    return abs(y) < 1.0 and pt > 0.2

In [14]:
h2_m_pt = root.TH2D(
    "h2_m_pt",
    "e^{+}e^{-}: M_{ee} vs p_{T};p_{T} (GeV/c);M_{ee} (GeV/c^{2})",
    200, 0.0, 10.0,
    1000, 0.0, 5.0
)

h_mult_final = root.TH1D("h_mult_final", "Final-state multiplicity;N_{final};Events", 20, -0.5, 19.5)
h_br_final   = root.TH1D("h_br_final",   "Final track branch br;br;Counts",            50, -0.5, 49.5)
h1_pt_original = root.TH1D("h1_pt_original", "Original pT distribution;p_{T} (GeV/c);Counts", 100, 0.0, 10.0)
h1_pt_mother_reweighted = root.TH1D("h1_pt_mother_reweighted", "Mother pT distribution (reweighted);p_{T} (GeV/c);Counts", 100, 0.0, 10.0)

In [15]:
import math
import ROOT as root

def CrystalBall2(x, par):
    # ROOT passes x as a "view" (cppyy.LowLevelView). Convert explicitly.
    x0 = float(x[0])

    N      = float(par[0])
    mu     = float(par[1])
    s      = float(par[2])
    n      = float(par[3])
    alpha2 = float(par[4])
    m      = float(par[5])
    beta   = float(par[6])

    if s == 0.0:
        return 0.0

    abs_alpha2 = abs(alpha2)
    abs_beta   = abs(beta)
    if abs_alpha2 == 0.0 or abs_beta == 0.0:
        return 0.0

    A = (n / abs_alpha2) ** n * math.exp(-0.5 * alpha2 * alpha2)
    B = n / abs_alpha2 - abs_alpha2

    C = (m / abs_beta) ** m * math.exp(-0.5 * beta * beta)
    D = m / abs_beta - abs_beta

    norm = (x0 - mu) / s

    if norm < -alpha2:
        return N * A * (B - norm) ** (-n)
    elif norm < beta:
        return N * math.exp(-0.5 * norm * norm)
    else:
        return N * C * (D + norm) ** (-m)

In [16]:

funSmearPt = root.TF1("funSmearPt","sqrt([0]*[0]*x*x+[1]*[1])",0.,10.)
mPtSmearPar = array('d', [6.0e-3, 8.3e-03, 0.000511])  # example parameters: [A, B, C] for sqrt((A*pT)^2 + B^2 + C^2)
funSmearPt.SetParameters(mPtSmearPar)
funSmearPt.SetNpx(10000)
funSmearPt.Print()


momShape = root.TF1("momShape",CrystalBall2,-1.,1,7)
#momShape->SetParameters(1., -1e-3, 0.01, 1.16, 1.84, 2.18, 1.92);//-3.9e-4
momShape.SetParameters(1., -1e-3, 0.01, 1.29, 1.75, 2.92, 1.84)
momShape.SetNpx(10000)

Formula based function:     funSmearPt 
           funSmearPt : sqrt([0]*[0]*x*x+[1]*[1]) Ndim= 1, Npar= 2, Number= 0 
 Formula expression: 
	sqrt([p0]*[p0]*x*x+[p1]*[p1]) 


In [17]:
c7 = root.TCanvas("c7", "pT smearing distribution", 800, 600)
momShape.Draw()
c7.Draw()

In [18]:
pt_smear_hist = root.TH2D("pt_smear_hist", "pT smearing distribution;p_{T}^{original} (GeV/c);p_{T}^{smeared} - p_{T}^{original} (GeV/c)", 100, 0.0, 10.0, 100, -0.5, 0.5)
def smear_pt(pt_orig):
    #pt is set
    epsig = funSmearPt.Eval(pt_orig)
    newpt  = pt_orig*(1. + momShape.GetRandom()*epsig/0.01)
    pt_smear_hist.Fill(pt_orig, newpt - pt_orig)
    return newpt
def smaer_p4(e):
    eppt = e.Pt()
    if eppt <= 0:
        return e  # no smearing for zero or negative pT
    epsig = funSmearPt.Eval(eppt)
    newpt  = eppt*(1. + momShape.GetRandom()*epsig/0.01)
    eta = e.Eta()
    phi = e.Phi()
    smeared_p4 = root.TLorentzVector()
    smeared_p4.SetPtEtaPhiM(newpt, eta, phi, e.M())
    return smeared_p4

In [19]:
nentries = int(T.GetEntries())
if nev_set is not None:
    nentries = min(nentries, int(nev_set))

for iev in range(nentries):
    T.GetEntry(iev)
    ev = T.MyEvent

    ntrk = int(ev.GetNEntries())

    mother_p4 = None
    finals = []

    for i in range(ntrk):
        trk = ev.GetWriteTrack(i)
        info = track_info(trk)

        if info["isFinal"] == 0:
            # primary mother (ParentIndex < 0)
            if mother_p4 is None and info["parent"] < 0:
                mother_p4 = info["p4"]
            continue

        finals.append(info)
        h_br_final.Fill(info["br"])

    h_mult_final.Fill(len(finals))

    eminus = [t for t in finals if t["pid"] == 11]
    eplus  = [t for t in finals if t["pid"] == -11]

    pt_mother = mother_p4.Pt() if mother_p4 is not None else None
    h1_pt_original.Fill(pt_mother if pt_mother is not None else -1)

    ### UPDATED: diagnostic fill uses pt_reweight(pt_mother)
    if pt_mother is not None:
        wpt = pt_reweight(pt_mother)
        if DO_PT_REWEIGHT:
            h1_pt_mother_reweighted.Fill(pt_mother, wpt)

    for e1 in eminus:

        if DO_SMAER_PT:
            #smarign momentum of first electron
            e1["p4"] = smaer_p4(e1["p4"])
        if DO_SINGLE_TRACK_CUT and not single_track_cut(e1["p4"]):
            continue
        for e2 in eplus:
            if DO_SMAER_PT:
                #smarign momentum of second electron
                e2["p4"] = smaer_p4(e2["p4"])
            if DO_SINGLE_TRACK_CUT and not single_track_cut(e2["p4"]):
                continue
            pair = e1["p4"] + e2["p4"]
            mee  = pair.M()
            pt_pair = pair.Pt()

            # file branching weights
            w = 1.

            ### UPDATED: TF1-based reweight (target/gen), using mother pT
            if DO_PT_REWEIGHT:
                if pt_mother is None:
                    continue
                w *= pt_reweight(pt_mother)

            h2_m_pt.Fill(pt_pair, mee, w)

    if iev > 0 and (iev % 1000000 == 0):
        print("Processed", iev, "/", nentries)

In [20]:
h1_pt_original.Scale(1.0 / nentries * (GEN_PT_MAX - GEN_PT_MIN) / h1_pt_original.GetBinWidth(1))
h1_pt_mother_reweighted.Scale(1.0 / nentries / h1_pt_mother_reweighted.GetBinWidth(1))

In [21]:
c3 = root.TCanvas("c3","c3",800,600)
h1_pt_original.Draw()
c3.Draw()
c4 = root.TCanvas("c4","c4",800,600)
h1_pt_mother_reweighted.Draw()
c4.Draw()

In [22]:
c1 = root.TCanvas("c1","c1",900,700)
h2_m_pt.Draw("COLZ")
c1.SetLogz(True)
c1.SaveAs("output/hists/mee_vs_pt.png")

c2 = root.TCanvas("c2","c2",800,600)
h_mult_final.Draw()
c2.SaveAs("output/hists/mult_final.png")

c3 = root.TCanvas("c3","c3",800,600)
h_br_final.Draw()
c3.SaveAs("output/hists/br_final.png")

In [23]:
c5 = root.TCanvas("c5","c5",800,600)
mass_hist = h2_m_pt.ProjectionY("mass_hist")
mass_hist.Scale(1./0.75 / nentries / mass_hist.GetBinWidth(1) * 2)  # normalize by number of events
mass_hist.Draw()
c5.Draw()

In [24]:
f_200 = root.TFile.Open("../dca/input/Star200.root","READ")
g200 = f_200.Get("Table 13").Get("Graph2D_y1;1")
if not g200:
    g200 = f_200.Get("Table 13").Get("Graph2D_y1")
if not g200:
    raise RuntimeError("Graph2D_y1 not found")

n = g200.GetN()
f_200.Close

gr_200 = root.TGraphErrors(n)
for i in range(n):
    m  = float(g200.GetX()[i])  # mass
    acc = 1.
    dm = float(g200.GetY()[i])  # dmass (usually bin width)
    val= float(g200.GetZ()[i])  # dN/dmass
    val_err = float(g200.GetErrorZ(i))  # error on dN/dmass
    gr_200.SetPoint(i, m, val/acc)
    gr_200.SetPointError(i, 0.5*dm, val_err/acc)  # x-error from bin width

In [25]:
c = root.TCanvas("c2","c2",900,650)
root.gPad.SetLogy()
gr_200.GetYaxis().SetRangeUser(9e-6, 20)
gr_200.SetMarkerStyle(20)
gr_200.SetMarkerColor(root.kGreen+2)
gr_200.SetLineColor(root.kGreen+2)
gr_200.SetTitle("Table 13; mass; dN/dmass")
gr_200.Draw("AP")
Format_Graph(mass_hist,Msize=1,Mcolor=root.kRed)
mass_hist.Draw("SAME")
c.Draw()


In [26]:
txt_path = "input/STAR_gdir_AuAu.txt"  # uploaded file :contentReference[oaicite:0]{index=0}

# pt-bin labels as they appear in the file
pt_bins = [
    ("1.0-1.5", "1-1.5GeV/c"),
    ("1.5-2.0", "1.5-2.0GeV/c"),
    ("2.0-2.5", "2.0-2.5GeV/c"),
    ("2.5-3.0", "2.5-3.0GeV/c"),
]

def _centers_to_edges(centers):
    """Build variable bin edges from bin centers (midpoint rule)."""
    c = centers
    if len(c) < 2:
        # fallback: +/- 0.5 around the single point
        return [c[0] - 0.5, c[0] + 0.5]
    edges = [c[0] - 0.5*(c[1]-c[0])]
    for i in range(len(c)-1):
        edges.append(0.5*(c[i] + c[i+1]))
    edges.append(c[-1] + 0.5*(c[-1]-c[-2]))
    return edges

def read_star_txt_make_hists(path):
    # storage: key -> list of (mee, val, statp)
    data = {k[0]: [] for k in pt_bins}
    current = None

    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            # detect which table we are in by the pt-range tag in header line
            if "d^2Nee/dM_{ee}dp_{T}" in line:
                current = None
                for lbl, tag in pt_bins:
                    if tag in line:
                        current = lbl
                        break
                continue

            # stop table on metadata lines
            if line.startswith("#:"):
                current = None
                continue

            # numeric rows: Mee  value  stat+  stat-  sys+  sys-
            if current is not None and (line[0].isdigit() or line[0] == "."):
                parts = line.split()
                if len(parts) >= 3:
                    mee   = float(parts[0])
                    val   = float(parts[1])
                    statp = float(parts[2])  # use stat+ as bin error
                    data[current].append((mee, val, statp))

    # build 4 TH1D
    hists = []
    for lbl, _tag in pt_bins:
        rows = data[lbl]
        if len(rows) == 0:
            print(f"WARNING: no rows found for pT bin {lbl}")
            continue

        centers = [r[0] for r in rows]
        edges   = _centers_to_edges(centers)
        nbins   = len(centers)

        h = root.TH1D(f"hMee_pt_{lbl.replace('.','p').replace('-','to')}",
                      f"STAR dielectron;M_{{ee}} (GeV/c^{{2}});d^{{2}}N_{{ee}}/dMdp_{{T}}",
                      nbins, array("d", edges))

        for i, (mee, val, statp) in enumerate(rows, start=1):
            h.SetBinContent(i, val)
            h.SetBinError(i, abs(statp))

        h.SetDirectory(0)  # keep histogram alive if file closes
        hists.append(h)

    return hists

hMee_1p0_1p5, hMee_1p5_2p0, hMee_2p0_2p5, hMee_2p5_3p0 = read_star_txt_make_hists(txt_path)

In [27]:
# quick check draw
c = root.TCanvas("c_star_txt", "STAR txt -> TH1D", 1200, 1200)
c.Divide(2,2)
mass_hists_pt = []
for i, h in enumerate([hMee_1p0_1p5, hMee_1p5_2p0, hMee_2p0_2p5, hMee_2p5_3p0], start=1):
    c.cd(i)
    h.Draw("E1")
    root.gPad.SetLogy()

    mass_hists_pt.append(h2_m_pt.ProjectionY(f"mass_hist_pt{i}",h2_m_pt.GetXaxis().FindBin(0.5+i*0.5),h2_m_pt.GetXaxis().FindBin(0.99+i*0.5)))
    mass_hists_pt[-1].Scale(1./0.75 / nentries / mass_hists_pt[-1].GetBinWidth(1) * 2 / 0.5)  # normalize by number of events
    mass_hists_pt[-1].Rebin(2)
    mass_hists_pt[-1].Scale(0.5)
    Format_Graph(mass_hists_pt[-1], Msize=1, Mcolor=root.kRed)
    if False: mass_hists_pt[-1].Scale(h.Integral(1,3) / mass_hists_pt[-1].Integral(1,3))
    mass_hists_pt[-1].Draw("Same")
    
c.Draw()

In [28]:
out = root.TFile(outfilename, "RECREATE")
h2_m_pt.Write()
h_mult_final.Write()
h_br_final.Write()
out.Close()

print("Wrote pair_hists.root")

Wrote pair_hists.root
